In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
%matplotlib inline

In [5]:
consumption = {}
consumption['dryer'] = np.array([900,1000,1200,1150,1000,930,720,900,850])/1000
consumption['dish_washer'] = np.array([1150,1280,900,990,720,340,850])/1000
consumption['washing_maschine'] = np.array([900,1000,1730,720,900,670,1150,840,750])/1000

prizes = [0.23, 0.20, 0.14, 0.19,  0.17, 0.17, 0.26, 0.19,0.12, 0.8, 0.14, 0.17, 0.14, 0.14, 0.19,0.12, 0.8, 0.14, 0.12, 0.8, 0.14, 0.17, 0.18, 0.21]
prizes = np.array([prizes, prizes, prizes, prizes, prizes, prizes]).flatten()
print(np.shape(prizes))

(144,)


In [23]:
# available data: dict with average comsumtion load curve of dryer, washing maschine, dish washer in 10 min steps
# engergy prizes in 10 minutes steps

def calculateCost(start_time, end_time, device_key):
    # either start_time, end_time in index format or convert
    
    runtime_steps = len(consumption[device_key])
    assert end_time - start_time >= runtime_steps
    latest_start_time = end_time - runtime_steps
    time_steps = (latest_start_time - start_time) +1
    results = np.zeros(time_steps)
    
    for i in range(time_steps):
        temp_start_time = start_time + i
        temp_sum = 0
        for j in range(runtime_steps):
            temp_sum = temp_sum + consumption[device_key][j]*prizes[temp_start_time + j]
        results[i] = temp_sum
        
    return start_time, results

def giveIdealTimes(start_time, result_vector):
    indexes = [i+ start_time for i, x in enumerate(result_vector) if x == min(result_vector)]
    return indexes

def timeToIndex(hour, minutes):
    assert minutes %10 == 0 and hour in range(24) and  minutes in range(60)
    index = hour*6 + minutes/10
    return int(index)

def indexToTime(index):
    hours_index = math.floor(index/6)
    hours =['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23'][hours_index]
    minutes = (index - hours_index*6)*10
    if minutes == 0:
        time = hours +':00  '
    else:
        time = hours +':'+str(minutes)+'  '
    return time


In [24]:
start_index = timeToIndex(13,20) # 08:20
end_index = timeToIndex(16,30) #16:30

[start_time, res] = calculateCost(start_index, end_index,'washing_maschine')

print(res)
ideal_indexes = giveIdealTimes(start_time, res)
print(ideal_indexes)
time = indexToTime(ideal_indexes[0])
print('Ideal time is at '+time)


[2.4117 2.4232 2.018  2.1778 2.4099 2.4711 2.8586 2.5546 2.4394 2.5891
 2.1128]
[82]
Ideal time is at 13:40  


In [26]:
times = [indexToTime(i+start_time) for i in range(len(res))]

df = pd.DataFrame(res, index=times, columns={'Prize'})
display(df)

,Prize
13:20,2.4117
13:30,2.4232
13:40,2.0180
13:50,2.1778
14:00,2.4099
14:10,2.4711
14:20,2.8586
14:30,2.5546
14:40,2.4394
14:50,2.5891
